# Convert the TensorFlow-Trained BERT Model to a PyTorch Model

In [1]:
import boto3
import sagemaker
import pandas as pd

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

# Install TensorFlow and Transformers

In [2]:
!pip install -q tensorflow==2.1.0 --upgrade --ignore-installed
!pip install -q torch==1.5.0 --upgrade --ignore-installed
!pip install -q transformers==2.8.0

ERROR: awscli 1.18.24 has requirement botocore==1.15.24, but you'll have botocore 1.17.15 which is incompatible.
ERROR: awscli 1.18.24 has requirement rsa<=3.5.0,>=3.1.2, but you'll have rsa 4.6 which is incompatible.


# Retrieve the Training Job Name

In [3]:
%store -r training_job_name

In [4]:
print('Previous training_job_name: {}'.format(training_job_name))

Previous training_job_name: tensorflow-training-2020-06-06-15-43-12-549


# Download the TensorFlow-Trained Model Locally

In [5]:
models_dir = './models'

In [6]:
# Download the model and output artifacts from AWS S3
!aws s3 cp s3://$bucket/$training_job_name/output/model.tar.gz $models_dir/model.tar.gz

download: s3://sagemaker-us-east-1-806570384721/tensorflow-training-2020-06-06-15-43-12-549/output/model.tar.gz to models/model.tar.gz


In [7]:
import tarfile
import pickle as pkl

#!ls -al ./models

tar = tarfile.open('{}/model.tar.gz'.format(models_dir))
tar.extractall(path=models_dir)
tar.close()

In [8]:
!ls -al $models_dir

total 487076
drwxrwxr-x 6 ec2-user ec2-user      4096 Jul  4 15:37 .
drwxrwxr-x 9 ec2-user ec2-user      4096 Jul  4 15:35 ..
drwxrwxr-x 2 ec2-user ec2-user      4096 Jul  2 17:51 .ipynb_checkpoints
-rw-rw-r-- 1 ec2-user ec2-user 498739800 Jun  6 16:48 model.tar.gz
drwxrwxr-x 2 ec2-user ec2-user      4096 Jul  2 17:51 pytorch
drwxr-xr-x 3 ec2-user ec2-user      4096 Jun  6 15:46 tensorflow
drwxr-xr-x 3 ec2-user ec2-user      4096 Jun  6 15:46 transformers


In [9]:
transformer_model_dir = '{}/transformers/fine-tuned/'.format(models_dir)

!ls -al $transformer_model_dir

total 261692
drwxr-xr-x 2 ec2-user ec2-user      4096 Jun  6 16:47 .
drwxr-xr-x 3 ec2-user ec2-user      4096 Jun  6 15:46 ..
-rw-r--r-- 1 ec2-user ec2-user      1358 Jun  6 16:47 config.json
-rw-r--r-- 1 ec2-user ec2-user 267959068 Jun  6 16:47 tf_model.h5


In [10]:
cat $transformer_model_dir/config.json

{
  "_num_labels": 5,
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "bad_words_ids": null,
  "bos_token_id": null,
  "decoder_start_token_id": null,
  "dim": 768,
  "do_sample": false,
  "dropout": 0.1,
  "early_stopping": false,
  "eos_token_id": null,
  "finetuning_task": null,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "is_decoder": false,
  "is_encoder_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4
  },
  "length_penalty": 1.0,
  "max_length": 20,
  "max_position_embeddings": 512,
  "min_length": 0,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "no_repeat_ngram_size": 0,
  "num_beams": 1,
  "num_return_sequences": 1,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true

In [11]:
!pip install torch==1.5.0
!pip install transformers==2.8.0 --ignore-installed

  Using cached transformers-2.8.0-py3-none-any.whl (563 kB)
  Using cached numpy-1.19.0-cp36-cp36m-manylinux2010_x86_64.whl (14.6 MB)
  Using cached filelock-3.0.12-py3-none-any.whl (7.6 kB)
  Using cached boto3-1.14.16-py2.py3-none-any.whl (128 kB)
Processing /home/ec2-user/.cache/pip/wheels/49/25/98/cdea9c79b2d9a22ccc59540b1784b67f06b633378e97f58da2/sacremoses-0.0.43-py3-none-any.whl
  Using cached dataclasses-0.7-py3-none-any.whl (18 kB)
  Using cached tqdm-4.47.0-py2.py3-none-any.whl (66 kB)
  Using cached sentencepiece-0.1.91-cp36-cp36m-manylinux1_x86_64.whl (1.1 MB)
  Using cached tokenizers-0.5.2-cp36-cp36m-manylinux1_x86_64.whl (3.7 MB)
  Using cached requests-2.24.0-py2.py3-none-any.whl (61 kB)
  Using cached regex-2020.6.8-cp36-cp36m-manylinux2010_x86_64.whl (660 kB)
  Using cached jmespath-0.10.0-py2.py3-none-any.whl (24 kB)
  Using cached botocore-1.17.16-py2.py3-none-any.whl (6.3 MB)
  Using cached s3transfer-0.3.3-py2.py3-none-any.whl (69 kB)
  Using cached six-1.15.0-py2

# Restart the Kernel

In [12]:
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

# Convert the TensorFlow Model to PyTorch

In [13]:
from transformers import DistilBertForSequenceClassification # PyTorch version

loaded_pytorch_model = DistilBertForSequenceClassification.from_pretrained(transformer_model_dir,
                                                                     id2label={
                                                                       0: 1,
                                                                       1: 2,
                                                                       2: 3,
                                                                       3: 4,
                                                                       4: 5
                                                                     },
                                                                     label2id={
                                                                       1: 0,
                                                                       2: 1,
                                                                       3: 2,
                                                                       4: 3,
                                                                       5: 4
                                                                     },
                                                                  from_tf=True)

In [14]:
print(type(loaded_pytorch_model))
print(loaded_pytorch_model)

<class 'transformers.modeling_distilbert.DistilBertForSequenceClassification'>
DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
   

In [15]:
from transformers import DistilBertTokenizer

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

inference_device = -1 # CPU: -1, GPU: 0
print('inference_device {}'.format(inference_device))

inference_device -1


In [16]:
from transformers import TextClassificationPipeline

inference_pipeline = TextClassificationPipeline(model=loaded_pytorch_model, 
                                                tokenizer=tokenizer,
                                                framework='pt',
                                                device=inference_device) 

In [17]:
review = """I loved it!  I will recommend this to everyone."""
print(review, inference_pipeline(review))

I loved it!  I will recommend this to everyone. [{'label': 4, 'score': 0.2661953}]


In [18]:
review = """Really bad.  I hope they don't make this anymore."""
print(review, inference_pipeline(review))

Really bad.  I hope they don't make this anymore. [{'label': 4, 'score': 0.2674279}]


# Save The Model with `.save_pretrained()`

In [40]:
pytorch_models_dir = './models/transformers/pytorch'

In [41]:
loaded_pytorch_model.save_pretrained(pytorch_models_dir)

In [42]:
!ls -al $pytorch_models_dir

total 261588
drwxrwxr-x 2 ec2-user ec2-user      4096 Jul  4 15:54 .
drwxr-xr-x 5 ec2-user ec2-user      4096 Jul  4 15:54 ..
-rw-rw-r-- 1 ec2-user ec2-user      1302 Jul  4 15:54 config.json
-rw-rw-r-- 1 ec2-user ec2-user 267852933 Jul  4 15:54 pytorch_model.bin


### Load to check it is working

In [43]:
pytorch_model = DistilBertForSequenceClassification.from_pretrained(pytorch_models_dir)

In [44]:
from transformers import TextClassificationPipeline

inference_pipeline = TextClassificationPipeline(model=pytorch_model, 
                                                tokenizer=tokenizer,
                                                framework='pt',
                                                device=inference_device) 

In [45]:
review = """I loved it!  I will recommend this to everyone."""
print(review, inference_pipeline(review))

I loved it!  I will recommend this to everyone. [{'label': 4, 'score': 0.2661953}]


# Save The Model with `torch.save()` as native PyTorch model

In [52]:
pytorch_model_path = './models/pytorch/{}'.format('pytorch_model.pt')

In [53]:
import torch
torch.save(loaded_pytorch_model.state_dict(), pytorch_model_path)

In [54]:
!ls -al $pytorch_model_path

-rw-rw-r-- 1 ec2-user ec2-user 267852933 Jul  4 15:58 ./models/pytorch/pytorch_model.pt
